<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/titanic_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Titanic
## Methodology for evaluating model performance


In [0]:
import pandas as pd 


url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/titanic_%20train.csv' 
titanic = pd.read_csv(url)

colunas = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

In [0]:
# filling in the gaps

titanic0 = titanic.copy().loc[titanic['Survived'] == 0]
for column in titanic.columns:
  if titanic0[column].dtype != 'object':
      titanic0[column].fillna(titanic0[column].mean(), inplace= True)
  else:
      titanic0[column] = titanic0[column].astype('str')
      titanic0[column].fillna(titanic0[column].mode(), inplace= True)

titanic1 = titanic.copy().loc[titanic['Survived'] == 1]
for column in titanic.columns:
  if titanic1[column].dtype != 'object':
      titanic1[column].fillna(titanic1[column].mean(), inplace= True)
  else:
      titanic1[column] = titanic1[column].astype('str')
      titanic1[column].fillna(titanic1[column].mode(), inplace= True)